In [107]:

import pandas as pd
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=2,
              batch_size=128, callbacks=callbacks_list)




In [109]:
notes = []
log = []

annees = [2002]
notes = []
for x in annees:
    notesAnnees = pd.read_pickle(r'../data/results/notes_' + str(x))
    notes += notesAnnees
notes = notes[1000:2000]
n_vocab = len(set(notes))

In [110]:
print('creating inputs')
network_input, network_output = prepare_sequences(notes, n_vocab)

print('creating model for training')
model = create_network(network_input, n_vocab)

print('training model')
train(model, network_input, network_output)

print('model trained')

creating inputs
creating model for training
training model
Epoch 1/2
8/8 [==============================] - 18s 2s/step - loss: 6.2690
Epoch 2/2
8/8 [==============================] - 13s 2s/step - loss: 6.0461
model trained


In [111]:
print('creating model for prediction')
print('be sure to have the correct path for weights...')
model = create_network(network_input, n_vocab)
model.load_weights('weights-improvement-02-6.0190-bigger.hdf5')

creating model for prediction
be sure to have the correct path for weights...


In [112]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 100 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]

generating notes


In [113]:
prediction_output

['B-3.F#3',
 'B-3.F#3',
 'B-3.F#3',
 'B-3.F#3',
 'B-3.F#3',
 'B-3.F#3',
 'B-3.F#3',
 'B-3.F#3',
 'B-3.F#3',
 'B-3.F#3']

# 

In [94]:
pattern

array([9.76635514e-01, 9.85981308e-01, 5.16355140e-01, 8.99532710e-01,
       9.64953271e-01, 4.27570093e-01, 7.87383178e-01, 9.67289720e-01,
       2.54672897e-01, 8.99532710e-01, 9.64953271e-01, 4.46261682e-01,
       4.64953271e-01, 2.47663551e-01, 3.73831776e-02, 7.54672897e-01,
       9.27570093e-01, 9.67289720e-01, 2.54672897e-01, 3.29439252e-01,
       9.64953271e-01, 4.27570093e-01, 9.90654206e-01, 4.27570093e-01,
       2.57009346e-01, 7.38317757e-01, 5.37383178e-02, 4.27570093e-01,
       9.64953271e-01, 4.64953271e-01, 2.45327103e-01, 7.57009346e-01,
       5.95794393e-01, 5.16355140e-01, 9.64953271e-01, 4.27570093e-01,
       7.47663551e-01, 3.94859813e-01, 3.27102804e-01, 9.64953271e-01,
       4.27570093e-01, 8.31775701e-01, 7.50000000e-01, 4.43925234e-01,
       3.69158879e-01, 7.50000000e-01, 4.27570093e-01, 9.64953271e-01,
       5.16355140e-01, 7.45327103e-01, 7.61682243e-01, 4.46261682e-01,
       9.64953271e-01, 9.90654206e-01, 7.24299065e-01, 3.55140187e-01,
      

In [11]:
network_input

array([[[0.17757009],
        [0.14252336],
        [0.27102804],
        ...,
        [0.22196262],
        [0.40654206],
        [0.85981308]],

       [[0.14252336],
        [0.27102804],
        [0.26869159],
        ...,
        [0.40654206],
        [0.85981308],
        [0.64953271]],

       [[0.27102804],
        [0.26869159],
        [0.17757009],
        ...,
        [0.85981308],
        [0.64953271],
        [0.67757009]],

       ...,

       [[0.83411215],
        [0.44859813],
        [0.89953271],
        ...,
        [0.46495327],
        [0.64485981],
        [0.39719626]],

       [[0.44859813],
        [0.89953271],
        [0.26635514],
        ...,
        [0.64485981],
        [0.39719626],
        [0.22429907]],

       [[0.89953271],
        [0.26635514],
        [0.78738318],
        ...,
        [0.39719626],
        [0.22429907],
        [0.14018692]]])

In [13]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []


In [15]:
int_to_note

{0: 'G3.E-3',
 1: 'C#4.C#3',
 2: 'B-5',
 3: 'G4.A4',
 4: 'E-5.G4.G5.A5.A4',
 5: 'A4.A3.G3.G4',
 6: 'B1.B-2.G#2',
 7: 'G5.G3',
 8: 'A4.G4.E-4',
 9: 'E-5.A5.G4.A4.G5',
 10: 'E-4',
 11: 'C#4.C#3.B-3.C3',
 12: 'F4.G4',
 13: 'E-4.G6',
 14: 'B-4.G5',
 15: 'G4.D4.G3',
 16: 'G#4.D4',
 17: 'F#4.A4',
 18: 'B4.G4',
 19: 'E-5.C#4.E4',
 20: 'C#5.E-4',
 21: 'E-4.A4.G4.G3.A3',
 22: 'D5.G4.G5',
 23: 'G#6',
 24: 'B-4',
 25: 'A3.C3.E3',
 26: 'G4.E-5',
 27: 'E-4.G3.G4.A3',
 28: 'G#5.G#4',
 29: 'B-2.B-1',
 30: 'A4.A3.G3',
 31: 'G3.E4.D4',
 32: 'E-5.G5.A4.A5.G4',
 33: 'E3.A2',
 34: 'B-3.D4.G3',
 35: 'E5',
 36: 'A1.A2.B2',
 37: 'F#1',
 38: 'D4.G#4.A3.G3',
 39: 'G#4.E-5.C#4',
 40: 'F4.G5',
 41: 'G1.A1',
 42: 'E-4.A4',
 43: 'E4.C#4.C#5.E-4',
 44: 'G5.D5.G4',
 45: 'A3.C3',
 46: 'G3.C3',
 47: 'B4',
 48: 'C5.C4',
 49: 'E5.A4',
 50: 'F4.B3',
 51: 'G3.D4.B-3',
 52: 'E-5.A4',
 53: 'C#4',
 54: 'E-4.B3',
 55: 'F4',
 56: 'C5.F4.G5',
 57: 'F5.D5.B-5',
 58: 'E4.G3',
 59: 'F4.C5.G5.F5.B4',
 60: 'F#6',
 61: 'F3.B-2.D3',
 

In [16]:
pattern


array([[0.46495327],
       [0.24766355],
       [0.03738318],
       [0.7546729 ],
       [0.92757009],
       [0.96728972],
       [0.2546729 ],
       [0.32943925],
       [0.96495327],
       [0.42757009],
       [0.99065421],
       [0.42757009],
       [0.25700935],
       [0.73831776],
       [0.05373832],
       [0.42757009],
       [0.96495327],
       [0.46495327],
       [0.2453271 ],
       [0.75700935],
       [0.59579439],
       [0.51635514],
       [0.96495327],
       [0.42757009],
       [0.74766355],
       [0.39485981],
       [0.3271028 ],
       [0.96495327],
       [0.42757009],
       [0.8317757 ],
       [0.75      ],
       [0.44392523],
       [0.36915888],
       [0.75      ],
       [0.42757009],
       [0.96495327],
       [0.51635514],
       [0.7453271 ],
       [0.76168224],
       [0.44626168],
       [0.96495327],
       [0.99065421],
       [0.72429907],
       [0.35514019],
       [0.96728972],
       [0.92990654],
       [0.35514019],
       [0.752

In [ ]:
# 

In [ ]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 100 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]

In [ ]:
prediction_output

# 

In [ ]:
# 

In [ ]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 100 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note)
                       for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = numpy.append(pattern,index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [ ]:
t = generate_notes(model, network_input, set(notes), n_vocab)

In [ ]:
t

# 

In [78]:
# 

creating model for prediction


ValueError: Cannot assign to variable dense_71/kernel:0 due to variable shape (256, 284) and value shape (256, 179) are incompatible

In [76]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 100 notes
for note_index in range(10):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]

generating notes


In [104]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note)
                       for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = numpy.append(pattern,index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [105]:
t = generate_notes(model, network_input, set(notes), n_vocab)

In [106]:
t

['F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#3.F#2',
 'F#

# 